#  Train a GPT-2 Text-Generating Model w/ GPU

This notebook is based on [Max Woolf](http://minimaxir.com)'s [notebook](https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce).

You can visit his [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use the notebook.

## CUDA setup

In [ ]:
!sudo apt-get install -y --no-install-recommends libnvinfer7=7.0.0-1+cuda10.0 \
    libnvinfer-dev=7.0.0-1+cuda10.0 \
    libnvinfer-plugin7=7.0.0-1+cuda10.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  libnvinfer-dev libnvinfer-plugin7 libnvinfer7
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 143 MB of archives.
After this operation, 520 MB of additional disk space will be used.
Get:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  libnvinfer7 7.0.0-1+cuda10.0 [69.6 MB]
Get:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  libnvinfer-dev 7.0.0-1+cuda10.0 [71.6 MB]
Get:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  libnvinfer-plugin7 7.0.0-1+cuda10.0 [2,109 kB]
Fetched 143 MB in 3s (56.3 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <

#  Installing required libraries


In [ ]:
%tensorflow_version 1.x
#!pip install tensorflow==1.15.2
!pip install tensorflow-gpu==1.15.2
!pip install -q gpt-2-simple==0.7.2

TensorFlow 1.x selected.
     |████████████████████████████████| 410.9 MB 33 kB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=9ebfc870ca8e1345bc279d13eebff3ed6b0d127abb2526746266478bbfac8302
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.6 requires tensorflow>=2.0.0, but you have tensorflow 1.15.2 which is incompatible.


In [ ]:
import gpt_2_simple as gpt2
from datetime import datetime

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## Verifying GPU

In [ ]:
!nvidia-smi

Sat Aug 14 08:31:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1883901346309836980
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 9809514574139687328
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15132320802690925497
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15964005991
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12539786915253490866
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model.
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [ ]:
gpt2.download_gpt2(model_name="355M")

Fetching checkpoint: 1.05Mit [00:00, 744Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 2.25Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 783Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [01:02, 22.7Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 706Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 2.29Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 3.44Mit/s]


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !unzip drive/MyDrive/Flute/data/dataset-gpt.zip -d data

If your input text dataset is massive (>100 MB), you may want to preencode and compress the dataset using `gpt2.encode_dataset(file_path)`. THe output is a compressed `.npz` file which will load much faster into the GPU for finetuning.

In [ ]:
# gpt2.encode_dataset("data/dataset-gpt/",
#                     out_path='data/compressed_data.npz',
#                     model_name="355M"
#                     )

Reading files


100%|██████████| 93826/93826 [10:53<00:00, 143.59it/s]


Writing data/compressed_data.npz


## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
#data_path = "drive/MyDrive/shared/GPT/data/compressed_netbeans_eclipse.npz"
#data_path = "drive/MyDrive/shared/GPT/data/compressed_9000.npz"
data_path = "drive/MyDrive/shared/GPT/data/compressed_ecnet_fold9_excluded.npz"
#data_path = "drive/MyDrive/shared/GPT/data/compressed_ecnet_fold0_to_2.npz"

In [ ]:
NUM_STEP = 50000

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=data_path,
              model_name='355M',
              steps=NUM_STEP,
              restore_from='fresh',
              run_name='run1',
              print_every=100,
              sample_every=1000,
              save_every=1000,
              overwrite=True,
              #use_memory_saving_gradients=True,
              #accumulate_gradients=1,
              #only_train_transformer_layers=True
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:02<00:00,  2.19s/it]


dataset has 24928809 tokens
Training...
[100 | 97.02] loss=1.52 avg=1.52
[200 | 185.10] loss=2.01 avg=1.77
[300 | 273.14] loss=1.40 avg=1.64
[400 | 361.20] loss=1.59 avg=1.63
[500 | 449.27] loss=1.68 avg=1.64
[600 | 537.34] loss=1.62 avg=1.64
[700 | 625.40] loss=1.78 avg=1.66
[800 | 713.44] loss=1.62 avg=1.65
[900 | 801.49] loss=1.17 avg=1.60
[1000 | 889.54] loss=1.36 avg=1.57
Saving checkpoint/run1/model-1000
======== SAMPLE 1 ========
){ return null; }}<|endoftext|>HtmlUtils {/** * @param page * @param image * @param text * @param parent */ public static Page getPage(Composite parent){ if (page == null){ return null; } Image image=page.getImage(); if (image == null){ return null; } String text=getText(page,page).contains(""); String text=getText(page,page.getFullTitle(),page.getFullEmail(),page.getFullTitle.count()); if (text != null){ return text; } Image image=getImage(text,page.getFullTitle(),page.getFullEmail(),<COMPOUND>text+1) + 1; String text=getText(page,page.getFullEmail(),p

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.tar` compressed file; you can download it and uncompress it locally.

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

In [ ]:
# import os
# import tarfile

# def get_tarfile_name(checkpoint_folder):
#     """Converts a folder path into a filename for a .tar archive"""
#     tarfile_name = checkpoint_folder.replace(os.path.sep, '_') + '.tar'

#     return tarfile_name

# checkpoint_folder = os.path.join('checkpoint', 'run1')

# file_path = get_tarfile_name(checkpoint_folder)

# with tarfile.open(file_path, 'w') as tar:
#     tar.add(checkpoint_folder)

## Load a Trained Model Checkpoint

Running the next cell will copy the `.tar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.